In [ ]:
# ------------------------------------------------------------
# 0) Install libraries
# ------------------------------------------------------------
!pip install lifelines

# ------------------------------------------------------------
# 1) Imports
# ------------------------------------------------------------
import os, random, copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from lifelines import CoxPHFitter, KaplanMeierFitter
from sklearn.model_selection import train_test_split

# ------------------------------------------------------------
# 2) Reproducibility
# ------------------------------------------------------------
RNG_SEED = 42
def seed_all(seed=RNG_SEED):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)

seed_all(RNG_SEED)

# ------------------------------------------------------------
# 3) Load data
# ------------------------------------------------------------
df = pd.read_csv("/content/drive/MyDrive/Paper(2025Dec)_SimulatedCVD/Data001_ReadyForCoxPH.csv")
for col in ["Unnamed: 0", "index"]:
    if col in df.columns:
        df = df.drop(columns=[col])
df_raw = copy.copy(df)

# Basic sanity checks (optional)
required_cols = ["cvd_time", "cvd_event", "IRSD_quintile", "Age", "smoking_status",
                 "AF", "CKD", "diabetes", "HbA1c", "eGFR", "SBP"]
missing = [c for c in required_cols if c not in df_raw.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 350.0/350.0 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 9.0 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=154a7da9b41900dfe0be0bfd209c43bc4b809c844c8526c852c4b484f7ef0cb1
  Stored in directory: /root/.cache/pip/wheels/50/37/21/0a719b9d89c635e89ff24bd93b862882ad675279552013b2fb
Successfully built autograd-gamma


In [ ]:
# ------------------------------------------------------------
# 4) Transform once: age centering
# ------------------------------------------------------------
df_raw = df_raw.copy()
df_raw["Age_c"] = df_raw["Age"] - 30.0

# Ensure types
for col in ["AF", "CKD", "diabetes", "cvd_event"]:
    df_raw[col] = df_raw[col].astype(int)

# ------------------------------------------------------------
# 5) Feature builder WITHOUT IRSD as a feature
# ------------------------------------------------------------
def build_X_no_irsd(df, drop_cols=None):
    df = df.copy()

    # smoking dummies (baseline = non-smoker)
    smoke = pd.get_dummies(df["smoking_status"], prefix="smoke").fillna(0)
    if "smoke_non" in smoke.columns:
        smoke = smoke.drop(columns=["smoke_non"])

    X = pd.concat(
        [
            df[["Age_c", "AF", "CKD", "diabetes", "HbA1c", "eGFR", "SBP"]],
            smoke,
        ],
        axis=1
    )

    # drop near-constant cols based on (training) df only
    if drop_cols is None:
        events = df["cvd_event"].astype(bool)
        drop_cols = []
        for c in X.columns:
            v_all = X[c].var()
            v_e   = X.loc[events, c].var() if events.any() else 0.0
            v_ne  = X.loc[~events, c].var() if (~events).any() else 0.0
            if (v_all < 1e-6) or (v_e < 1e-6) or (v_ne < 1e-6):
                drop_cols.append(c)

    return X.drop(columns=drop_cols, errors="ignore"), drop_cols



In [ ]:
# ------------------------------------------------------------
# 6) Calibration helpers (censoring-aware observed risk via KM within bins)
# ------------------------------------------------------------
def predict_risk_at_t(cph, X, t):
    S_t = cph.predict_survival_function(X, times=[t]).iloc[0].to_numpy(float)
    return np.clip(1.0 - S_t, 0.0, 1.0)

def km_event_risk_at_t(times, events, t):
    """
    Estimate P(T <= t) = 1 - S_KM(t) in a group.
    Returns NaN if not enough data.
    """
    times = np.asarray(times, dtype=float)
    events = np.asarray(events, dtype=int)

    if len(times) < 10:
        return np.nan

    kmf = KaplanMeierFitter()
    try:
        kmf.fit(durations=times, event_observed=events)
        s_t = float(kmf.survival_function_at_times(t).iloc[0])
        return float(np.clip(1.0 - s_t, 0.0, 1.0))
    except Exception:
        return np.nan

def calibration_d21_by_qbins(risk_t, y_time, y_event, t, K=20):
    """
    Bin by predicted risk quantiles, then compute:
      r_bar: mean predicted risk per bin
      y_bar: KM-estimated observed event risk by t per bin
    Fit y_bar ≈ a * r_bar (through origin), return D21 = |1 - a|
    """
    tmp = pd.DataFrame({
        "risk":  np.asarray(risk_t, dtype=float),
        "time":  np.asarray(y_time, dtype=float),
        "event": np.asarray(y_event, dtype=int),
    }).dropna()

    if tmp.shape[0] < 50:
        return np.nan, np.nan, np.array([]), np.array([])

    tmp = tmp.sort_values("risk").reset_index(drop=True)
    tmp["bin"] = pd.qcut(tmp["risk"], q=K, duplicates="drop")

    r_bar, y_bar = [], []
    for _, g in tmp.groupby("bin", observed=False):
        r_mean = float(g["risk"].mean())
        y_km = km_event_risk_at_t(g["time"].to_numpy(float), g["event"].to_numpy(int), t)
        if np.isfinite(r_mean) and np.isfinite(y_km):
            r_bar.append(r_mean)
            y_bar.append(y_km)

    r_bar = np.asarray(r_bar, dtype=float)
    y_bar = np.asarray(y_bar, dtype=float)

    if len(r_bar) < 3 or float(np.dot(r_bar, r_bar)) == 0.0:
        return np.nan, np.nan, r_bar, y_bar

    a = float(np.dot(r_bar, y_bar) / np.dot(r_bar, r_bar))
    d21 = float(abs(1.0 - a))
    return a, d21, r_bar, y_bar

# ------------------------------------------------------------
# 7) Settings for calibration
# ------------------------------------------------------------
t = 4.88
K = 20
penalizer = 0.05

In [ ]:
# ------------------------------------------------------------
# 8) BASELINE comparator:
#    Train on ALL IRSD (but WITHOUT IRSD as a feature)
#    Then evaluate calibration (D21) separately within each IRSD quintile.
# ------------------------------------------------------------
X_all, drop_cols_all = build_X_no_irsd(df_raw, drop_cols=None)
surv_all = pd.concat(
    [df_raw[["cvd_time", "cvd_event", "IRSD_quintile"]].reset_index(drop=True),
     X_all.reset_index(drop=True)],
    axis=1
).dropna()

cph_baseline = CoxPHFitter(penalizer=penalizer)
cph_baseline.fit(surv_all.drop(columns=["IRSD_quintile"]), duration_col="cvd_time", event_col="cvd_event")

baseline_rows = []
for q in sorted(surv_all["IRSD_quintile"].dropna().unique()):
    q = int(q)
    g = surv_all[surv_all["IRSD_quintile"] == q].copy()
    if g.shape[0] < 200:
        baseline_rows.append((q, np.nan, np.nan, g.shape[0]))
        continue

    y_time = g["cvd_time"].to_numpy(float)
    y_event = g["cvd_event"].to_numpy(int)
    Xg = g.drop(columns=["cvd_time", "cvd_event", "IRSD_quintile"])

    risk_t = predict_risk_at_t(cph_baseline, Xg, t)
    a, d21, _, _ = calibration_d21_by_qbins(risk_t, y_time, y_event, t, K=K)
    baseline_rows.append((q, a, d21, g.shape[0]))

baseline_df = pd.DataFrame(baseline_rows, columns=["IRSD_quintile", "a_baseline", "D21_baseline", "n_test"])
print("\n=== Baseline: train on ALL IRSD (no IRSD feature), evaluate within each IRSD ===")
print(baseline_df.sort_values("IRSD_quintile"))


=== Baseline: train on ALL IRSD (no IRSD feature), evaluate within each IRSD ===
   IRSD_quintile  a_baseline  D21_baseline  n_test
0              1    1.167178      0.167178   10640
1              2    1.254565      0.254565    8055
2              3    1.221137      0.221137   11941
3              4    1.166861      0.166861    8495
4              5    1.111325      0.111325   10869


In [ ]:
# ------------------------------------------------------------
# 9) INTERNAL–EXTERNAL (leave-one-IRSD-out):
#    For each q:
#      Train on IRSD != q (no IRSD feature)
#      Test on IRSD == q
#      Compute D21 on that held-out stratum
# ------------------------------------------------------------
loo_rows = []

for q in sorted(df_raw["IRSD_quintile"].dropna().unique()):
    q = int(q)

    train_q = df_raw[df_raw["IRSD_quintile"] != q].copy()
    test_q  = df_raw[df_raw["IRSD_quintile"] == q].copy()

    # Build training design matrix (defines drop_cols for this run)
    X_tr, drop_cols = build_X_no_irsd(train_q, drop_cols=None)
    train_surv = pd.concat(
        [train_q[["cvd_time", "cvd_event"]].reset_index(drop=True),
         X_tr.reset_index(drop=True)],
        axis=1
    ).dropna()

    # Fit Cox on training data
    cph = CoxPHFitter(penalizer=penalizer)
    cph.fit(train_surv, duration_col="cvd_time", event_col="cvd_event")

    # Build testing design matrix (use training-derived drop_cols)
    X_te, _ = build_X_no_irsd(test_q, drop_cols=drop_cols)
    test_surv = pd.concat(
        [test_q[["cvd_time", "cvd_event"]].reset_index(drop=True),
         X_te.reset_index(drop=True)],
        axis=1
    ).dropna()

    if test_surv.shape[0] < 200:
        loo_rows.append((q, np.nan, np.nan, test_surv.shape[0]))
        continue

    y_time  = test_surv["cvd_time"].to_numpy(float)
    y_event = test_surv["cvd_event"].to_numpy(int)
    X_te_final = test_surv.drop(columns=["cvd_time", "cvd_event"])

    risk_t = predict_risk_at_t(cph, X_te_final, t)
    a, d21, _, _ = calibration_d21_by_qbins(risk_t, y_time, y_event, t, K=K)

    loo_rows.append((q, a, d21, test_surv.shape[0]))

loo_df = pd.DataFrame(loo_rows, columns=["IRSD_quintile", "a_loo", "D21_loo", "n_test"])
print("\n=== Leave-one-IRSD-out: train on others (no IRSD feature), test on held-out IRSD ===")
print(loo_df.sort_values("IRSD_quintile"))


=== Leave-one-IRSD-out: train on others (no IRSD feature), test on held-out IRSD ===
   IRSD_quintile     a_loo   D21_loo  n_test
0              1  1.162889  0.162889   10640
1              2  1.273879  0.273879    8055
2              3  1.238324  0.238324   11941
3              4  1.161915  0.161915    8495
4              5  1.087939  0.087939   10869
